In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import tqdm as tqdm

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
input_size=28
sequence_length=28
num_layers=2
hidden_size=300
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=2

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        #h0:(2,n,300)
        #x:(n,28,28)

        # Forward propagate LSTM
        out, _ = self.rnn(x, h0)
        
        #out:(n,28,300)
        out = out.reshape(out.shape[0], -1)
        #out:(n,28*300)
        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out


In [5]:
# Load Data
train_dataset = datasets.MNIST(root="data1/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="data1/", train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
model = RNN( input_size, hidden_size, num_layers, num_classes).to(device)

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate((train_loader)):
        # Get data to cuda if possible
        
        # data:(64,1,28,28)
        data = data.to(device=device).squeeze(1)
        # data:(64,28,28)
        targets = targets.to(device=device)
        
        
        # forward
        scores = model(data)
        
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()

In [10]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 96.660004
Accuracy on test set: 96.51
